# Algorithms

## Dependence

In [179]:
# dependence
import os
import random
# env
CD = '/Users/yangmo/Downloads'

## Tools

In [163]:
class DataLoader():
    def __init__(self, fileName, numLines = None, processLine=lambda x:x):
        self.data = None
        self.numLines = numLines
        self.file = os.path.join(CD,fileName)
        self.processLine = processLine
        
    def read(self):
        print('Loading data ...')
        out = []
        with open(self.file, 'r') as f:
            lines = f.readlines()
            for num,line in enumerate(lines):
                line = self.processLine(line)
                out.append(line)
                if self.numLines:
                    if num == self.numLines-1:
                        break
        self.data = out
        print(f'Total {len(out)} lines read')
        print('-'*50)

In [162]:
class Algorithm():
    def __init__(self,dataLoader):
        self.dataLoader = dataLoader
        self.data = None
        self.res = None
    
    def readData(self): # read data from dataLoader
        dataLoader.read()
        self.data = dataLoader.data
    
    def algorithm(self):
        pass

    def run(self, data=None):# given data, use data to run algorithm, if not, load data from dataLoader first
        if data:
            self.data = data
            self.res = self.algorithm()
        else:
            self.readData()
            self.res = self.algorithm()
        print(f'result:{self.res}')

In [29]:
# Test DataLoader class
processLine = lambda x: int(x.strip())  # define the way to precessing line for dataLoader
arg = {'fileName':'test.txt','numLines':10,'processLine':processLine } # prepare arg for dataLoader
dataLoader = DataLoader(**arg)
dataLoader.read()
print(dataLoader.out)

Loading data ...
Total 10 lines read
[54044, 14108, 79294, 29649, 25260, 60660, 2995, 53777, 49689, 9083]


# PART 1 Divide and Conquer
* Week 2 Counting Inversions (Merget Sort)
* Week 3 Quick Sort
* Week 4 Randomized Selection 

In [5]:
##Template

def recursion(problem):
    if condition: return T(1)
    result_1 = recursion(sub_problem_1)
    result_2 = recursion(sub_problem_2)
    ...
    result_n = recursion(sub_problem_2)
    total_result = conbine(results)
    return total_result 

### Week 2 Counting Inversions (Merget Sort)

In [156]:
class CI(Algorithm):
    def __init__(self,dataLoader):
        super().__init__(dataLoader)
        self.res = None
        
    def algorithm(self):
        def sortNcount(A, n):
            if n == 1: return (A,0)
            left,right = A[:n//2],A[n//2:]
            (B,x) = sortNcount(left,len(left))
            (C,y) = sortNcount(right,len(right))
            #(D,z) = mergeNcountSplitInv(A,n)
            i,j = 0,0
            z = 0
            D = [0]*n
            for k in range(n):
                #print(f'i:{i}; j:{j}; len(B):{len(B)}; len(C):{len(C)}')
                if B[i]<C[j]:
                    D[k]=B[i]
                    i+=1
                else:
                    D[k]=C[j]
                    j+=1
                    z+=(len(B)-i)
                if i==len(B):
                    D[k+1:]=C[j:]
                    break
                elif j==len(C):
                    D[k+1:]=B[i:]
                    break
            return D,x+y+z
        return sortNcount(self.data,len(self.data))
         
    def run(self, data=None):# given data, use data to run algorithm, if not, load data from dataLoader first
        if data:
            self.data = data
            self.res = self.algorithm()
        else:
            self.readData()
            self.res = self.algorithm()
        print(f'result: {self.res[1]}')

In [157]:
## test CI
processLine = lambda x: int(x.strip())
arg = {'fileName':'test.txt','numLines':100,'processLine':processLine }
dataLoader = DataLoader(**arg)
ci = CI(dataLoader)
ci.run()

Loading data ...
Total 100 lines read
--------------------------------------------------
result: 2461


### Week 3 Quick Sort

In [151]:
class QS(Algorithm):
    def __init__(self,dataLoader=None):
        super().__init__(dataLoader)
        self.assignment = 0
        self.methods = [self.first,self.last, self.median] #self.median]
    
    def first(self,r,l):
        return r
    
    def last(self,r,l):
        return l
        
    def median(self,l,r):
        candidate = [(l,self.data[l]),((r-l)//2+l, self.data[(r-l)//2+l]),(r, self.data[r])]
        candidate.sort(key=lambda x:x[1])
        return candidate[1][0]
        
    def algorithm(self,method):
        def quickSort(l,r):
            if r-l<1: return
            self.assignment+=(r-l)
            c = method(l,r)
            p = self.partition(l,r,c)
            quickSort(l,max(l,p-1))
            quickSort(min(r,p+1),r)
        quickSort(0,len(self.data)-1)
        return self.data
    
    def partition(self,l,r,c): # inplace O(n) swap partition
            self.data[c],self.data[l]=self.data[l],self.data[c]
            p = self.data[l]  # how to chose pivot
            i = l+1
            for j in range(i,r+1):
                if self.data[j]<p:
                    self.data[i],self.data[j]=self.data[j],self.data[i]
                    i+=1
            self.data[l],self.data[i-1] = self.data[i-1],self.data[l]
            return i-1
        
    def run(self, data=None):
        if data:
            self.data = data
            self.algorithm(self.first)
        else:
            for method in self.methods:
                self.assignment = 0
                self.readData()
                self.algorithm(method)
                print(f'* {method.__name__}:{self.assignment}')
                print('='*50,end='\n'*2)


In [152]:
# Test quickSort
processLine = lambda x: int(x.strip())
arg = {'fileName':'QuickSort.txt','numLines':None,'processLine':processLine }
dataLoader = DataLoader(**arg)
qs = QS(dataLoader)
qs.run()

Loading data ...
Total 10000 lines read
--------------------------------------------------
* first:162085

Loading data ...
Total 10000 lines read
--------------------------------------------------
* last:164123

Loading data ...
Total 10000 lines read
--------------------------------------------------
* median:138382



### Week 4 Randomized Selection

In [218]:
class MinCut(Algorithm):
    def __init__(self, dataLoader):
        super().__init__(dataLoader)
        self.candidates=[]
    
    def preprocess(self):
        out = {}
        for line in self.data:
            out[line[0]]=line[1:]
        self.data = out
        
    def algorithm(self):
        while len(self.data)>2:
            ## random find one edge
            pool=[]
            for key,array in self.data.items():
                for ele in array:
                    pool.append((key,ele))
            edge=random.choice(pool)
            ## merge two vertex
            head,tail = edge
            self.data[head]= self.data[head]+self.data[tail]
            del(self.data[tail])
            for vertex in self.data:
                self.data[vertex]=[x if x!=tail else head for x in self.data[vertex]]
            # remove self-loop
            self.data[head]=[x for x in self.data[head] if x!=head and x!=tail]
        num = 0
        for vertex in self.data:
            num+=len(self.data[vertex])
        self.candidates.append(num/2)
        
    def run(self, times):
        for i in range(times):
            self.readData()
            self.preprocess()
            self.algorithm()
            
        print(f'Cuts:{self.candidates}, minCut:{min(self.candidates)}')
        

In [222]:
## test read graph
processLine = lambda line : list(map(int,line.split()))
arg = {'fileName':'kargerMinCut.txt','numLines':None,'processLine':processLine }
dataLoader = DataLoader(**arg)
mc = MinCut(dataLoader)
mc.run(2)

Loading data ...
Total 200 lines read
--------------------------------------------------
Loading data ...
Total 200 lines read
--------------------------------------------------
Cuts:[31.0, 44.0], minCut:31.0
